#XGBRegressor

In [24]:
#libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

In [25]:
df = pd.read_csv('/content/drive/MyDrive/ProjectCdac/TerraTrend-HousePricePrediction_DataExploration.csv')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141304 entries, 0 to 141303
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Price (in rupees)    141304 non-null  float64
 1   location             141304 non-null  object 
 2   Carpet Area in sqft  141304 non-null  float64
 3   Status               141304 non-null  int64  
 4   Transaction          141304 non-null  float64
 5   Furnishing           141304 non-null  float64
 6   facing               141304 non-null  int64  
 7   Bathroom             141304 non-null  int64  
 8   Balcony              141304 non-null  int64  
 9   Ownership            141304 non-null  float64
 10  Final Amount         141304 non-null  float64
 11  BHK                  141304 non-null  float64
 12  Super Area in sqft   141304 non-null  float64
dtypes: float64(8), int64(4), object(1)
memory usage: 14.0+ MB


In [27]:
X = df.drop('Final Amount', axis=1)  # Replace 'target' with your regression label
y = df['Final Amount']

In [28]:
X_ohe = pd.get_dummies(X)

In [29]:
#Define XGBRegressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_ohe, y, test_size=0.2, random_state=42)

In [31]:
#Define XGBRegressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

In [32]:
#Set up hyperparameter grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 6]
}

In [33]:
# Perform GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)


In [34]:
# Fit the model
grid_search.fit(X_train, y_train)

# Predict on test set
y_pred = grid_search.predict(X_test)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [35]:
# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) # Calculate RMSE by taking the square root of MSE
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [36]:
# Output results
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (Negative MSE):", grid_search.best_score_)
print(f"Test MSE: {mse:.2f}")
print(f"Test RMSE: {rmse:.2f}")
print(f"Test MAE: {mae:.2f}")
print(f"Test R² Score: {r2:.2f}")

Best Parameters: {'learning_rate': 0.2, 'max_depth': 6, 'n_estimators': 200}
Best CV Score (Negative MSE): -36.12324695900885
Test MSE: 29.19
Test RMSE: 5.40
Test MAE: 1.99
Test R² Score: 0.99


In [37]:
# Training metrics
y_train_pred = grid_search.predict(X_train)

train_mse = mean_squared_error(Y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(Y_train, y_train_pred)

test_rmse = np.sqrt(mse) # Use the previously calculated mse from the test set
test_r2 = r2_score(Y_test, y_pred) # Use the previously calculated y_pred and Y_test

print("Training RMSE:", train_rmse)
print("Training R²:", train_r2)

print("Testing RMSE:", test_rmse)
print("Testing R²:", test_r2)

Training RMSE: 4.437439217806557
Training R²: 0.9944178666445441
Testing RMSE: 5.403010808096706
Testing R²: 0.9916189859154745


In [38]:

# Training R² > Testing R²

# Training RMSE < Testing RMSE

# Model is clearly overfitting.